True

In [1]:
# Add the following parameter in .env file and then start running the program
# TAVILY_API_KEY=tvly-dev-IkhRjiLSssdodsijfldskjp

# Ref Link For LangChain Tool Integrations: https://docs.langchain.com/oss/python/integrations/tools

from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

True

In [2]:
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents.openai_functions_agent.base import create_openai_functions_agent
from langchain.agents import AgentExecutor

In [3]:
# 1) LLM (tool-calling capable)
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)


In [4]:
# 2) Tool: Tavily web search
search = TavilySearchResults(max_results=5)  # tweak: search_depth="basic"/"advanced"
tools = [search]


In [5]:
# 3) Prompt for the functions agent
prompt = ChatPromptTemplate.from_messages(
    [
        ("system",
         "You are a helpful research assistant. Use tools when needed and always cite sources with links."),
        MessagesPlaceholder("agent_scratchpad"),
        ("human", "{input}")
    ]
)


In [6]:
# 4) Build the agent and executor (LangChain-only)
agent = create_openai_functions_agent(llm=llm, tools=tools, prompt=prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [7]:
# 5) Run a query that requires search
query = "Give 4 crisp bullets on the latest progress of India’s semiconductor policy and include source links."
result = agent_executor.invoke({"input": query})
print("\n=== FINAL ===\n", result["output"])



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'India semiconductor policy latest news 2023'}`


[{'title': 'Notices | ISM - India Semiconductor Mission', 'url': 'https://ism.gov.in/notifications/notices', 'content': 'Government invites applications for setting up Semiconductor and Display Fabs as per the Modified Semicon India Programme from 1st June 2023', 'score': 0.8430832}, {'title': "Semiconductor Clusters in the Making: India's Push for Global ...", 'url': 'https://www.csis.org/analysis/semiconductor-clusters-making-indias-push-global-competitiveness', 'content': 'A2: As per industry estimates, the domestic semiconductor market was valued at approximately $38 billion in 2023, is projected to reach $45–50 billion in 2024–2025, and could expand to $100–110 billion by 2030. Achieving this growth trajectory will require both geographic specialization and robust policy frameworks. India’s emerging semiconductor clusters are the result 

In [13]:
# 5) Run a query that requires search
query = "Who is the Prime Minister of India in Jan 2025?"
result = agent_executor.invoke({"input": query})
print("\n=== FINAL ===\n", result["output"])



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'Prime Minister of India January 2025'}`


[{'title': 'List of all Prime Ministers of India (1947-2025) - Jagran Josh', 'url': 'https://www.jagranjosh.com/general-knowledge/list-of-all-prime-ministers-of-india-1473165149-1', 'content': "Currently, Narendra Modi is the Prime Minister of India, having been in office since 2014. He is the 14th Prime Minister of India who had three consecutive terms on the post.He took the Prime Minister's Oath on June 9, 2024.\n\nThis article provides a complete list of all the Prime Ministers of India from 1947 to 2025, along with their contributions to the country’s growth. The list reflects India's democratic resilience and its leaders' varied contributions to the nation's progress. [...] Position Name Ministries\nPrime Minister Shri Narendra Modi Ministry of Personnel, Public Grievances and Pensions, Department of Atomic Energy, Department of Space\nCabinet

# Combining Two Tools (Tavily + Calculator)

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents.openai_functions_agent.base import create_openai_functions_agent
from langchain.agents import AgentExecutor

from langchain_core.tools import tool

@tool
def pct(value: float, percent: float) -> float:
    """Return `percent` percent of `value`."""
    return value * percent / 100.0




# --- LLM (tool-calling capable) ---
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# --- Tools ---
search = TavilySearchResults(max_results=5, search_depth="advanced")  # uses TAVILY_API_KEY

tools = [search, pct]  

# --- Prompt: encourage “search → compute → cite” ---
prompt = ChatPromptTemplate.from_messages([
    ("system",
     "You are a precise research assistant. "
     "Use the web search for time-sensitive facts. "
     "Use the Python tool for calculations. "
     "Always include source links for facts from the web. "
     "If a tool errors, say so and do not fabricate."),
    MessagesPlaceholder("agent_scratchpad"),
    ("human", "{input}")
])

# --- Agent & Executor (LangChain-only) ---
agent = create_openai_functions_agent(llm=llm, tools=tools, prompt=prompt)
agent_exec = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True, max_iterations=10)

# --- Example query that forces both tools ---
q = (
    "Find the latest India population estimate from reputable sources, "
    "then compute 2% of it and report the result with the source link."
)

out = agent_exec.invoke({"input": q})
print("\n=== FINAL ===\n", out["output"])




> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'latest India population estimate 2023'}`


[{'title': "India to overtake China as world's most populous country in April ...", 'url': 'https://www.un.org/en/desa/india-overtake-china-world-most-populous-country-april-2023-united-nations-projects', 'content': 'UN logo\n© Smith Mehta/unsplash.\n\n# India to overtake China as world’s most populous country in April 2023, United Nations projects\n\n24 April 2023 - China will soon cede its long-held status as the world’s most populous country. By the end of this month, India’s population is expected to reach 1,425,775,850 people, matching and then surpassing the population of mainland China. This forecast is based on the latest United Nations estimates and projections of the global population. [...] According to Policy Brief No. 153 from the United Nations Department of Economic and Social Affairs (UN DESA), entitled “India overtakes China as the

In [21]:
out = agent_exec.invoke({"input": "My total marks is 80 out of 100. Whats my percentage?"})
print("\n=== FINAL ===\n", out["output"])



> Entering new AgentExecutor chain...

Invoking: `pct` with `{'value': 80, 'percent': 100}`


80.0
Invoking: `pct` with `{'value': 80, 'percent': 100}`


80.0Your percentage is 80%. This is calculated by taking your total marks (80) and dividing it by the maximum marks (100), then multiplying by 100. 

\[
\text{Percentage} = \left( \frac{80}{100} \right) \times 100 = 80\%
\]

> Finished chain.

=== FINAL ===
 Your percentage is 80%. This is calculated by taking your total marks (80) and dividing it by the maximum marks (100), then multiplying by 100. 

\[
\text{Percentage} = \left( \frac{80}{100} \right) \times 100 = 80\%
\]
